# Resolución de sistemas lineales
<!--
Generalidades:
 - Intro: importancia del problema, etc.
 - Nomenclatura, unicidad de la solución, condicionamiento
 - Métodos de resolución: operaciones elementales, overview métodos directos

Eliminacion de gauss:
Descomposición LU: Doolittle's / Choleski

Simétric/banded cases:

Pivoting

Matrix inversion
Métodos iterativos:
 - Intro
 - Gauss-Seidel
 - Gradiente Conjugado
 - 
-->

# Sistemas de ecuaciones lineales - generalidades

## Introducción

En este capítulo estudiaremos la solución de $n$ ecuaciones algebraicas lineales con $n$ incógnitas. Este es un tema de gran importancia en cálculo numérico. La razón es que los sistemas de ecuaciones que surgen orgánicamente de problemas físicos suelen ser muy grandes, y su solución a priori consume muchos recursos computacionales. Suele ser posible reducir los requisitos de almacenamiento y el tiempo de ejecución aprovechando las propiedades de cada matriz de coeficientes. De ahí que existan muchos algoritmos dedicados a la resolución de grandes sistemas de ecuaciones, cada uno de ellos adaptado a una forma particular de la matriz de coeficientes (simétrica, con bandas, dispersa, etc.).

## Notación

Un sistema de ecuaciones algebraicas tiene la forma:

$$
\begin{aligned}
A_{11} x_1+A_{12} x_2+\cdots+A_{1 n} x_n=& b_1 \\
A_{21} x_1+A_{22} x_2+\cdots+A_{2 n} x_n=& b_2 \\
& \vdots \\
A_{n 1} x_1+A_{n 2} x_2+\cdots+A_{n n} x_n=& b_n
\end{aligned}
$$
donde se conocen los coeficientes $A_{ij}$ y las constantes $b_j$. Las incógnitas son los valores $x_i$. En notación matricial, este sistema de ecuaciones se puede escribir como:

$$
\left[\begin{array}{cccc}
A_{11} & A_{12} & \cdots & A_{1 n} \\
A_{21} & A_{22} & \cdots & A_{2 n} \\
\vdots & \vdots & \ddots & \vdots \\
A_{n 1} & A_{n 2} & \cdots & A_{n n}
\end{array}\right]\left[\begin{array}{c}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{array}\right]=\left[\begin{array}{c}
b_1 \\
b_2 \\
\vdots \\
b_n
\end{array}\right]
$$
o simplemente:

$$
\mathbf{A} \mathbf{x} = \mathbf{b}
$$

Una representación especialmente útil de las ecuaciones para fines computacionales es la *matriz de coeficientes aumentada*, que se obtiene uniendo el vector de términos independientes $\mathbf{b}$ a la matriz de coeficientes $\mathbf{A}$ de la siguiente manera:

$$
[\mathbf{A} \mid \mathbf{b}]=\left[\begin{array}{cccc|c}
A_{11} & A_{12} & \cdots & A_{1 n} & b_1 \\
A_{21} & A_{22} & \cdots & A_{2 n} & b_2 \\
\vdots & \vdots & \ddots & \vdots & \vdots \\
A_{n 1} & A_{n 2} & \cdots & A_{n 3} & b_n
\end{array}\right]
$$

## Unicidad 

Un sistema de $n$ ecuaciones lineales con $n$ incógnitas tiene solución única siempre que el determinante de la matriz de coeficientes no sea singular; es decir, $\left| \mathbf{A} \right| \neq 0$. Las filas y columnas de una matriz no singular son linealmente independientes en el sentido de que ninguna fila (o columna) es una combinación lineal de otras filas (o columnas).
Si la matriz de coeficientes es singular, las ecuaciones pueden tener un número infinito de soluciones o ninguna, dependiendo del vector de términos independientes. Como ejemplo, tomemos las ecuaciones

$$
2x + y = 3 \quad 4x + 2y = 6
$$

Dado que la segunda ecuación puede obtenerse multiplicando la primera por dos, cualquier combinación de $x$ e $y$ que satisfaga la primera ecuación es también solución de la segunda. El número de estas combinaciones es infinito. En cambio, las ecuaciones

$$
2x + y = 3 \quad 4x + 2y = 0
$$

no tienen solución porque la segunda ecuación, que es equivalente a $2x + y = 0$, contradice a la primera. Por lo tanto, cualquier solución que satisfaga una ecuación no puede satisfacer la otra.

## Sistemas mal condicionados

Recuerda que, cuando el determinante de la matriz de coeficientes $\left| \mathbf{A} \right|$ no es 0 pero es muy pequeño (comparado con una norma de la matriz), se dice que el sistema está mal condicionado. Si es así, pequeños cambios en los datos (matriz de coeficientes o vector de incógnitas) originarán cambios significativos en la solución (vector de incógnitas). Formalmente, el condicionamiento se estudia a partir de lo que se conoce como *número de condición*, $\kappa$:

$$
\kappa \left( \mathbf{A} \right) = \left\| \mathbf{A} \right\| \cdot \left\| \mathbf{A}^{-1} \right\|
$$
si este número es próximo a 1, la matriz está bien condicionada. Recuerda también que el número de condición no es único, depende de la elección de la norma. Observa que el cálculo preciso del número de condición involucra el cálculo de la inversa de la matriz de coeficientes, lo cual es muy costoso computacionalmente. En la práctica, muchas veces se estima el número de condición a partir del determinante de la matriz $\mathbf{A}$, adimensionalizado con algún elemento de la matriz. 

Un sistema de ecuaciones mal condicionado puede generar soluciones no fiables. Los errores de redondeo en la resolución pueden alterar significativamente los datos y, por ende, la solución.


## Clasificación de los métodos de resolución

Existen dos clases de métodos para resolver sistemas de ecuaciones lineales: los **métodos directos** y los **métodos iterativos**. La característica común de los métodos directos es que transforman las ecuaciones originales en ecuaciones equivalentes (ecuaciones que tienen la misma solución) que pueden resolverse más fácilmente. La transformación se realiza aplicando las siguientes **tres operaciones**. Estas operaciones, denominadas elementales, no modifican la solución, pero pueden afectar al determinante de la matriz de coeficientes, como se indica entre paréntesis.
1.	Intercambio de dos ecuaciones (cambia el signo de $\left| \mathbf{A} \right|$)
2.	Multiplicar una ecuación por una constante no nula (multiplica $\left| \mathbf{A} \right|$ por la misma constante)
3.	Multiplicar una ecuación por una constante no nula y luego se la resta a otra ecuación (no modifica $\left| \mathbf{A} \right|$)


In [32]:
import numpy as np

def swap_rows(A, i , j):
    '''intercambia la fija j por la i 
    en la matriz de coeficientes A
    de un sistema lienal
    return una nueva matriz A con las filas intercambiadad
    pero con la matriz A original sin modificar'''

    A_copy = np.copy(A)

    ai = np.copy(A[i])
    

    A_copy[i] = A_copy[j]

    A_copy[j] = ai


    return A_copy


def k_row(A: np.array, i: int, k: float):

    '''miltiplicar la fila i de la matriz A
    por un valor k que puede ser un entero, fraccion...
    '''

    A_copy = np.copy(A)
    
    A_copy[i] = A_copy[i] * float(k)

    return A_copy

def row_lineal_comb(A, i, j, k):

    A_copy = np.copy(A)
    ai = np.copy(A[i])

    A_copy[j] = A_copy[j] - ai * k

    return A_copy



In [28]:
arr = np.array([[1,2,3], [4, 5, 6]])

print(arr[1] * 1/2)

[2.  2.5 3. ]


In [ ]:
A = np.array([[1, 2, 3],
             [3, 4, 4],
             [4, 5, 3]], dtype = float)

A_swap = swap_rows(A, 0, 1)
A_k = k_row(A, 0, 1/2)
A_cl = row_lineal_comb(A, 0, 1, 2)
#print(A_swap)
print()
print(A_k)
print()
print(A)
print()
print(A_cl)


[[0.5 1.  1.5]
 [3.  4.  4. ]
 [4.  5.  3. ]]

[[1. 2. 3.]
 [3. 4. 4.]
 [4. 5. 3.]]

[[ 1.  2.  3.]
 [ 1.  0. -2.]
 [ 4.  5.  3.]]


Los métodos iterativos (o indirectos) comienzan con una solución provisional $\mathbf{x}_0$, que será refinada de forma progresiva hasta que se alcance un determinado criterio de convergencia. Los métodos iterativos suelen ser menos eficientes que sus homólogos directos debido al gran número de iteraciones necesarias. Sin embargo, tienen importantes ventajas computacionales si la matriz de coeficientes es muy grande y está poco poblada (matriz *sparse*, la mayoría de sus coeficientes son 0).

### Resumen de los métodos directos
Hay muchísimos métodos directos para resolver sistemas lineales, pero aquí solo estudiaremos dos de los más importantes. La siguiente tabla los recoge, junto a la *forma final* del sistema a resolver tras realizar operaciones.

<center>

|      Método          | Forma inicial | Forma final |
|----------------------|---------------|-------------|
| Eliminación de Gauss | $\mathbf{A} \mathbf{x} = \mathbf{b}$ | $\mathbf{U} \mathbf{x} = \mathbf{c}$ |
| Descomposición LU	   | $\mathbf{A} \mathbf{x} = \mathbf{b}$ | $\mathbf{LU} \mathbf{x} = \mathbf{b}$ |

</center>


En la tabla anterior, $\mathbf{U}$ representa una matriz triangular superior, y $\mathbf{L}$ es una matriz triangular inferior. Una matriz cuadrada se llama *triangular* si solo contiene ceros a un lado de la diagonal principal. Así, una matriz triangular superior de $3 \times 3$ tiene la forma:

$$
\mathbf{U}=\left[\begin{array}{ccc}
U_{11} & U_{12} & U_{13} \\
0 & U_{22} & U_{23} \\
0 & 0 & U_{33}
\end{array}\right]
$$
 
y una matriz triangular inferior de $3 \times 3$ es:

$$
\mathbf{L}=\left[\begin{array}{ccc}
L_{11} & 0 & 0 \\
L_{21} & L_{22} & 0 \\
L_{31} & L_{32} & L_{33}
\end{array}\right]
$$

Las matrices triangulares desempeñan un papel importante en el álgebra lineal, ya que simplifican muchos cálculos. Por ejemplo, consideremos las ecuaciones: 

$$
\begin{aligned}
L_{11} x_1 &=c_1 \\
L_{21} x_1+L_{22} x_2 &=c_2 \\
L_{31} x_1+L_{32} x_2+L_{33} x_3 &=c_3
\end{aligned}
$$

Si resolvemos las ecuaciones hacia adelante, empezando por la primera ecuación, los cálculos son muy fáciles, porque cada ecuación contiene solo una incógnita nueva. La solución quedaría:

$$
\begin{aligned}
x_1 &=c_1 / L_{11} \\
x_2 &=\left(c_2-L_{21} x_1\right) / L_{22} \\
x_3 &=\left(c_3-L_{31} x_1-L_{32} x_2\right) / L_{33}
\end{aligned}
$$

Este procedimiento se conoce como *sustitución hacia delante* (*forward substitution*). De manera análoga, el sistema $\mathbf{U} \mathbf{x} = \mathbf{c}$ hallado en la eliminación de Gauss puede resolverse fácilmente mediante la *sustitución hacia atrás* (*backward substitution*), que comienza con la última ecuación y avanza hacia atrás.
El sistema $\mathbf{LU} \mathbf{x} = \mathbf{b}$ asociado a la descomposición LU, también pueden resolverse rápidamente si lo sustituimos por dos sistemas equivalentes: $\mathbf{L} \mathbf{y} = \mathbf{b}$ y $\mathbf{U} \mathbf{x} = \mathbf{y}$. Ahora $\mathbf{L} \mathbf{y} = \mathbf{b}$ puede resolverse mediante sustitución hacia delante, y a continuación el sistema $\mathbf{U} \mathbf{x} = \mathbf{y}$ se resuelve mediante sustitución hacia atrás.

**Ejercicio 1 -** Resuelve el sistema de ecuaciones $\mathbf{A} \mathbf{x} = \mathbf{b}$, donde 
$$
\mathbf{A}=\left[\begin{array}{rrr}
8 & -6 & 2 \\
-4 & 11 & -7 \\
4 & -7 & 6
\end{array}\right] \quad \mathbf{b}=\left[\begin{array}{r}
28 \\
-40 \\
33
\end{array}\right]
$$
sabiendo que la descomposición LU de la matriz de coeficientes es (compruébalo):
$$
\mathbf{A}=\mathbf{L} \mathbf{U}=\left[\begin{array}{rrr}
2 & 0 & 0 \\
-1 & 2 & 0 \\
1 & -1 & 1
\end{array}\right]\left[\begin{array}{rrr}
4 & -3 & 1 \\
0 & 4 & -3 \\
0 & 0 & 2
\end{array}\right]
$$